<a href="https://colab.research.google.com/github/Vignesh-P-C/fake-news-detection-transformers/blob/main/06_final_training_and_inference.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# First Training Run (Sanity Check)

**Project:** Fake News Detection using Transformers  
**Goal:** Verify end-to-end training works and loss decreases


In [ ]:
import pandas as pd
import torch
from sklearn.model_selection import train_test_split
from transformers import (
    AutoTokenizer,
    DistilBertForSequenceClassification,
    Trainer,
    TrainingArguments
)

MODEL_NAME = "distilbert-base-uncased"
MAX_LENGTH = 256

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

In [ ]:
fake_df = pd.read_csv(
    "Fake.csv",
    engine="python",
    sep=",",
    quotechar='"',
    escapechar="\\",
    on_bad_lines="skip"
)

true_df = pd.read_csv(
    "True.csv",
    engine="python",
    sep=",",
    quotechar='"',
    escapechar="\\",
    on_bad_lines="skip"
)

fake_df["label"] = 0
true_df["label"] = 1

df = pd.concat([fake_df, true_df], ignore_index=True)
df = df.sample(frac=1, random_state=42).reset_index(drop=True)

# FORCE correct types
texts = df["text"].astype(str).tolist()
labels = df["label"].tolist()

In [ ]:
type(texts), type(texts[0])

In [ ]:
train_texts, val_texts, train_labels, val_labels = train_test_split(
    texts,
    labels,
    test_size=0.2,
    random_state=42,
    stratify=labels
)

In [ ]:
assert isinstance(train_texts, list)
assert isinstance(train_texts[0], str)

In [ ]:
def tokenize_texts(texts):
    return tokenizer(
        texts,
        truncation=True,
        padding="max_length",
        max_length=MAX_LENGTH
    )

train_encodings = tokenize_texts(train_texts)
val_encodings = tokenize_texts(val_texts)


In [ ]:
class NewsDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {k: torch.tensor(v[idx]) for k, v in self.encodings.items()}
        item["labels"] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

In [ ]:
train_dataset = NewsDataset(train_encodings, train_labels)
val_dataset = NewsDataset(val_encodings, val_labels)

In [ ]:
model = DistilBertForSequenceClassification.from_pretrained(
    MODEL_NAME,
    num_labels=2
)

In [ ]:
training_args = TrainingArguments(
    output_dir="./results",
    eval_strategy="epoch",
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=1,
    learning_rate=2e-5,
    weight_decay=0.01,
    logging_steps=100,
    save_strategy="no",
    report_to="none"
)

In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
)

trainer.train()

In [ ]:
import numpy as np
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    preds = np.argmax(logits, axis=1)

    precision, recall, f1, _ = precision_recall_fscore_support(
        labels, preds, average="binary"
    )
    accuracy = accuracy_score(labels, preds)

    return {
        "accuracy": accuracy,
        "precision": precision,
        "recall": recall,
        "f1": f1
    }

In [ ]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics
)

In [ ]:
eval_results = trainer.evaluate()
eval_results

In [ ]:
len(set(train_texts).intersection(set(val_texts)))

In [ ]:
len(val_texts)

In [ ]:
print("Train size:", len(train_texts))
print("Val size:", len(val_texts))

print("Overlap:", len(set(train_texts).intersection(set(val_texts))))


In [ ]:
print(train_labels[:20])
print(val_labels[:20])

In [ ]:
predictions = trainer.predict(val_dataset)

logits = predictions.predictions
labels = predictions.label_ids

probs = torch.softmax(torch.tensor(logits), dim=1)
pred_classes = torch.argmax(probs, dim=1)

In [ ]:
confidences = probs.max(dim=1).values.numpy()

print("Min confidence:", confidences.min())
print("Mean confidence:", confidences.mean())
print("Max confidence:", confidences.max())

In [ ]:
import numpy as np

lowest_conf_indices = np.argsort(confidences)[:5]

for idx in lowest_conf_indices:
    print("Confidence:", confidences[idx])
    print("True label:", labels[idx])
    print("Predicted:", pred_classes[idx].item())
    print("Text snippet:", val_texts[idx][:200])
    print("-" * 50)

In [ ]:
num_train_epochs=3,

In [ ]:
save_strategy="epoch",

In [ ]:
load_best_model_at_end=True,
metric_for_best_model="f1",
greater_is_better=True,

In [ ]:
training_args = TrainingArguments(
    output_dir="./results",
    eval_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    metric_for_best_model="f1",
    greater_is_better=True,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    learning_rate=2e-5,
    weight_decay=0.01,
    logging_steps=100,
    report_to="none"
)

In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics
)

trainer.train()

In [ ]:
trainer.save_model("final_model")
tokenizer.save_pretrained("final_model")

In [ ]:
from transformers import DistilBertForSequenceClassification, AutoTokenizer

loaded_model = DistilBertForSequenceClassification.from_pretrained("final_model")
loaded_tokenizer = AutoTokenizer.from_pretrained("final_model")

print("Model loaded successfully.")

In [ ]:
import torch

def predict_news(text):
    inputs = loaded_tokenizer(
        text,
        truncation=True,
        padding="max_length",
        max_length=256,
        return_tensors="pt"
    )

    with torch.no_grad():
        outputs = loaded_model(**inputs)
        probs = torch.softmax(outputs.logits, dim=1)
        pred = torch.argmax(probs, dim=1).item()

    label = "REAL" if pred == 1 else "FAKE"
    confidence = probs[0][pred].item()

    return {
        "label": label,
        "confidence": confidence
    }

In [ ]:
predict_news("The president announced new economic reforms today.")